In [1]:
import pandas as pd
import csv
import collections
import re
import pickle
import numpy as np

In [2]:
fileNeop = "../data/RTRT_NEOPLASI_corr.csv"
fileIsto = "../data/ISTOLOGIE_corr.csv" 
fileIcdo = "icdo-3-dict-en.p"

fileSaveCorpus = "corpusTemporalBreastLung-multiclass/corpusTemporal.p"
fileSaveValues = "corpusTemporalBreastLung-multiclass/valuesTemporalSede1.p"
fileSaveTrainWordVectors = "corpusTemporalBreastLung-multiclass/trainWordVectors.txt"

In [3]:
numbers = {
    340:26,
    341:139,
    342:11,
    343:78,
    349:191,
    501:13,
    502:36,
    503:10,
    504:63,
    505:21,
    508:62,
    509:292,
}

In [4]:
encoding = 'iso-8859-1'

In [5]:
icdo = pickle.load(open(fileIcdo, 'rb'))

# Load neop

In [6]:
dfNeop = pd.read_csv(fileNeop, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False,
                     usecols = ['id_neopl', 'sede_icdo3',
                                'morfologia_icdo3', 'inserimento_data_neoplasi'],
                     dtype={'id_neopl':pd.np.float64, 'sede_icdo3':pd.np.str, 
                            'morfologia_icdo3':pd.np.str, 'inserimento_data_neoplasi':pd.np.str},
                    #parse_dates=['inserimento_data_neoplasi'], 
                    #date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
                    )

In [7]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [8]:
dfNeop.dtypes

id_neopl                     float64
inserimento_data_neoplasi     object
sede_icdo3                    object
morfologia_icdo3              object
dtype: object

In [9]:
dfNeop = dfNeop.dropna('index')

In [10]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [11]:
dfNeop.dtypes

id_neopl                     float64
inserimento_data_neoplasi     object
sede_icdo3                    object
morfologia_icdo3              object
dtype: object

In [12]:
dfNeop['inserimento_data_neoplasi'] =  pd.to_datetime(dfNeop['inserimento_data_neoplasi'], format='%Y-%m-%d %H:%M:%S')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [14]:
dfNeop.dtypes

id_neopl                            float64
inserimento_data_neoplasi    datetime64[ns]
sede_icdo3                           object
morfologia_icdo3                     object
dtype: object

In [15]:
dfNeop = dfNeop.sort_values(by=['inserimento_data_neoplasi'])

In [16]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
107276,108110.0,1990-04-20 00:00:00,C443,80003
78576,79172.0,1990-04-20 00:00:00,C341,80003
78595,79191.0,1990-04-24 00:00:00,C329,80703
78602,79198.0,1990-04-24 00:00:00,C509,85203
78601,79197.0,1990-04-24 00:00:00,C710,94003
...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001
295873,346866.0,2014-03-10 12:26:26,C169,81403


remove invalid sites

In [17]:
dfNeop['correctSede'] = dfNeop.sede_icdo3.map(lambda s: len(s)==4 and s[0] == "C" and int(s[1:3]) in icdo.keys() and s[3] in map(str, range(10)))

In [18]:
dfNeop = dfNeop[dfNeop.correctSede]

In [19]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede
107276,108110.0,1990-04-20 00:00:00,C443,80003,True
78576,79172.0,1990-04-20 00:00:00,C341,80003,True
78595,79191.0,1990-04-24 00:00:00,C329,80703,True
78602,79198.0,1990-04-24 00:00:00,C509,85203,True
78601,79197.0,1990-04-24 00:00:00,C710,94003,True
...,...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002,True
309849,346856.0,2014-02-14 11:53:40,C424,97651,True
309850,346857.0,2014-02-14 12:00:33,C619,80001,True
295873,346866.0,2014-03-10 12:26:26,C169,81403,True


In [20]:
dfNeop['y'] = dfNeop.sede_icdo3.map(lambda s: int(s[1:3]))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
dfNeop['ys'] = dfNeop.sede_icdo3.map(lambda s: int(s[3]))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede,y,ys
107276,108110.0,1990-04-20 00:00:00,C443,80003,True,44,3
78576,79172.0,1990-04-20 00:00:00,C341,80003,True,34,1
78595,79191.0,1990-04-24 00:00:00,C329,80703,True,32,9
78602,79198.0,1990-04-24 00:00:00,C509,85203,True,50,9
78601,79197.0,1990-04-24 00:00:00,C710,94003,True,71,0
...,...,...,...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002,True,50,9
309849,346856.0,2014-02-14 11:53:40,C424,97651,True,42,4
309850,346857.0,2014-02-14 12:00:33,C619,80001,True,61,9
295873,346866.0,2014-03-10 12:26:26,C169,81403,True,16,9


In [23]:
dfNeop['correctFilter'] = dfNeop.apply(lambda s: (s.y==34 and s.ys in [0,1,2,3,9]) or (s.y==50 and s.ys in [1,2,3,4,5,8,9]), axis=1)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
dfNeop = dfNeop[dfNeop.correctFilter]

In [25]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede,y,ys,correctFilter
78576,79172.0,1990-04-20 00:00:00,C341,80003,True,34,1,True
78602,79198.0,1990-04-24 00:00:00,C509,85203,True,50,9,True
78596,79192.0,1990-04-24 00:00:00,C509,85003,True,50,9,True
78592,79188.0,1990-04-24 00:00:00,C509,85223,True,50,9,True
78590,79186.0,1990-04-24 00:00:00,C349,80103,True,34,9,True
...,...,...,...,...,...,...,...,...
309842,346847.0,2014-01-24 12:24:11,C349,80003,True,34,9,True
309844,346849.0,2014-01-24 14:11:22,C349,80463,True,34,9,True
309846,346852.0,2014-01-27 11:19:52,C349,80003,True,34,9,True
309847,346854.0,2014-02-13 08:51:32,C509,85003,True,50,9,True


# Load isto

In [26]:
dfIsto = pd.read_csv(fileIsto, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False, 
                     usecols = ['notizie', 'diagnosi', 'macroscopia', 'id_neopl', 'id_fonte'],
                     dtype={'id_neopl':pd.np.float64, 'id_fonte':pd.np.float64})

In [27]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
0,1.0,NaN,NaN,NaN,NaN
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.
2,3.0,NaN,NaN,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM."
3,4.0,NaN,NaN,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...
4,5.0,NaN,NaN,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,NaN
...,...,...,...,...,...
1592380,1792447.0,NaN,NaN,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...
1592381,1792448.0,NaN,NaN,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...
1592383,1792450.0,NaN,NaN,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...


In [28]:
nullToEmpty = lambda s: "" if pd.isnull(s) else str(s)
dfIsto['notizie'] = dfIsto.notizie.map(nullToEmpty)
dfIsto['diagnosi'] = dfIsto.diagnosi.map(nullToEmpty)
dfIsto['macroscopia'] = dfIsto.macroscopia.map(nullToEmpty)

In [29]:
dfIsto = dfIsto.dropna('index')

In [30]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril..."
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.
...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...


In [31]:
dfIsto['text'] = dfIsto[['notizie' ,'diagnosi', 'macroscopia']].apply(lambda t: " ".join(t).upper(), axis=1)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
dfIsto['text'] = dfIsto['text'].astype('unicode')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
dfIsto['text'] = dfIsto['text'].apply(lambda t: " ".join(re.findall(r"\w+|[^\w\s]", t, re.UNICODE)))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...
...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...


In [35]:
dfIsto['correctText'] = dfIsto.text.map(lambda t: len(t) > 0)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
dfIsto = dfIsto[dfIsto.correctText]

In [37]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


In [38]:
dfIsto = dfIsto.drop_duplicates(['text'])

In [39]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


In [40]:
dfIsto = dfIsto.drop_duplicates(['id_neopl'], keep=False)

In [41]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
684,685.0,341032.0,,CARCINOMA BASOCELLULARE; MARGINI DI SEZIONE LI...,"1-2) Losanga di cute di cm 2.5, lesione irrego...",CARCINOMA BASOCELLULARE ; MARGINI DI SEZIONE L...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


# Merge

In [42]:
dfMerge = pd.merge(dfIsto, dfNeop, on='id_neopl', validate="one_to_one")

In [43]:
dfMerge

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede,y,ys,correctFilter
0,3594.0,304840.0,"TUMEFAZIONI SOLIDE LOGGIA PAROTIDEA SX, LA MAG...",TUMORE DI WARTHIN.,,"TUMEFAZIONI SOLIDE LOGGIA PAROTIDEA SX , LA MA...",True,2012-02-23 13:32:58,C349,80413,True,34,9,True
1,5346.0,195328.0,,CARCINOMA PAPILLARE NEL MATERIALE RELATIVO A L...,Quattro vetrini.,CARCINOMA PAPILLARE NEL MATERIALE RELATIVO A L...,True,2006-11-02 00:00:00,C349,80503,True,34,9,True
2,6908.0,327407.0,NODULO SOSPETTO Q.I.I. SX,CARCINOMA INFILTRANTE DELLA MAMMELLA,,NODULO SOSPETTO Q . I . I . SX CARCINOMA INFIL...,True,2012-06-18 19:47:46,C509,80003,True,50,9,True
3,8227.0,194715.0,CARCINOMA MAMMELLA DESTRA Q.S.I./Q.S.E. NEL 19...,MAMMELLA DESTRA (RIF. N. 3211/99): CARCINOMA D...,,CARCINOMA MAMMELLA DESTRA Q . S . I . / Q . S ...,True,2006-11-02 00:00:00,C509,80003,True,50,9,True
4,8261.0,155346.0,CA. MAMMELLA DETRA Q.S.E. DIAMETRO 20 MM: AMPI...,CARCINOMA DUTTALE INFILTRANTE NOS G2. INVASION...,1-2) LIMITE DI SEZIONE CHIRURGICA VERSO IL CAP...,CA . MAMMELLA DETRA Q . S . E . DIAMETRO 20 MM...,True,2006-04-03 00:00:00,C504,85003,True,50,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13920,1792278.0,339899.0,,Mammella sn: CARCINOMA DUTTALE IN SITU DI BASS...,Q.S.C. mammella sn cm 6x4x2 con adesa losanga ...,MAMMELLA SN : CARCINOMA DUTTALE IN SITU DI BAS...,True,2012-10-01 11:12:06,C509,85002,True,50,9,True
13921,1792315.0,339487.0,,CARCINOMA DUTTALE INFILTRANTE (N.O.S.) CON ESP...,Mammella dx cm 18x15x6 con tessuto adiposo asc...,CARCINOMA DUTTALE INFILTRANTE ( N . O . S . ) ...,True,2012-09-26 10:28:46,C508,85003,True,50,8,True
13922,1792325.0,337658.0,,CARCINOMA DUTTALE INFILTRANTE (N.O.S.) (G2) DE...,Quadrante Infero Centrale mammella sn cm 8x4x...,CARCINOMA DUTTALE INFILTRANTE ( N . O . S . ) ...,True,2012-09-10 16:46:54,C509,85003,True,50,9,True
13923,1792341.0,340648.0,,CARCINOMA DUTTALE INFILTRANTE (N.O.S.) (G3) DE...,Quadrante supero-esterno mammella sn cm 10x5x4...,CARCINOMA DUTTALE INFILTRANTE ( N . O . S . ) ...,True,2012-10-05 12:30:35,C504,85003,True,50,4,True


# Unlabelled

In [44]:
dfUn = pd.read_csv(fileIsto, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False, 
                     usecols = ['notizie', 'diagnosi', 'macroscopia', 'id_neopl', 'id_fonte'],
                     dtype={'id_neopl':pd.np.float64, 'id_fonte':pd.np.float64})

In [45]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
0,1.0,NaN,NaN,NaN,NaN
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.
2,3.0,NaN,NaN,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM."
3,4.0,NaN,NaN,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...
4,5.0,NaN,NaN,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,NaN
...,...,...,...,...,...
1592380,1792447.0,NaN,NaN,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...
1592381,1792448.0,NaN,NaN,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...
1592383,1792450.0,NaN,NaN,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...


In [46]:
dfUn = dfUn[~dfUn.id_fonte.isin(dfMerge.id_fonte)]

In [47]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
0,1.0,NaN,NaN,NaN,NaN
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.
2,3.0,NaN,NaN,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM."
3,4.0,NaN,NaN,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...
4,5.0,NaN,NaN,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,NaN
...,...,...,...,...,...
1592380,1792447.0,NaN,NaN,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...
1592381,1792448.0,NaN,NaN,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...
1592383,1792450.0,NaN,NaN,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...


In [48]:
nullToEmpty = lambda s: "" if pd.isnull(s) else str(s)
dfUn['notizie'] = dfUn.notizie.map(nullToEmpty)
dfUn['diagnosi'] = dfUn.diagnosi.map(nullToEmpty)
dfUn['macroscopia'] = dfUn.macroscopia.map(nullToEmpty)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [49]:
dfUn['text'] = dfUn[['notizie' ,'diagnosi', 'macroscopia']].apply(lambda t: " ".join(t).upper(), axis=1)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
dfUn['text'] = dfUn['text'].astype('unicode')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
dfUn['text'] = dfUn['text'].apply(lambda t: " ".join(re.findall(r"\w+|[^\w\s]", t, re.UNICODE)))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
dfUn['correctText'] = dfUn.text.map(lambda t: len(t) > 0)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
dfUn = dfUn[dfUn.correctText]

In [54]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLI...,True
2,3.0,NaN,,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM.",ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGM...,True
3,4.0,NaN,,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...,FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI A...,True
4,5.0,NaN,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
5,6.0,NaN,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO (vedi es...,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI E...,True
...,...,...,...,...,...,...,...
1592380,1792447.0,NaN,,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...,"RESIDUE , MINIME ESPRESSIONI DI MALIGNITA ' EP...",True
1592381,1792448.0,NaN,,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,True
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...,E . R . : TESSUTO PROSTATICO RESIDUO CON CARAT...,True
1592383,1792450.0,NaN,,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...,CARCINOMA A CELLULE SQUAMOSE ( G2 ) CON ESPRES...,True


In [55]:
dfUn = dfUn[~dfUn.text.isin(dfMerge.text)]

In [56]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLI...,True
2,3.0,NaN,,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM.",ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGM...,True
3,4.0,NaN,,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...,FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI A...,True
4,5.0,NaN,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
5,6.0,NaN,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO (vedi es...,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI E...,True
...,...,...,...,...,...,...,...
1592380,1792447.0,NaN,,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...,"RESIDUE , MINIME ESPRESSIONI DI MALIGNITA ' EP...",True
1592381,1792448.0,NaN,,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,True
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...,E . R . : TESSUTO PROSTATICO RESIDUO CON CARAT...,True
1592383,1792450.0,NaN,,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...,CARCINOMA A CELLULE SQUAMOSE ( G2 ) CON ESPRES...,True


# Final

In [57]:
dfFinal = dfMerge[['inserimento_data_neoplasi','text', 'y', 'ys']]

In [58]:
dfFinal = dfFinal.rename(columns={"inserimento_data_neoplasi": "data"})

In [59]:
dfFinal = dfFinal.sort_values(by=['data'])

In [60]:
dfFinal['yf'] = dfFinal.apply(lambda s: int(str(s.y) + str(s.ys)) , axis=1)

In [61]:
dfFinal

,data,text,y,ys,yf
7959,1996-03-13 00:00:00,CARCINOMA DUTTALE INFILTRANTE ( NOS ) ( G2 ) D...,50,9,509
150,1999-03-22 00:00:00,LINFOMA MAMMELLA DESTRA . LINFOMA A GRANDI CEL...,50,9,509
9321,2000-05-30 00:00:00,MANTENUTA RISERVA TEMPORANEA DEL PEZZO OPERATO...,34,1,341
9318,2000-05-30 00:00:00,CONGELATORE NR 46250 : NEOPLASIA MALIGNA NON U...,34,9,349
9317,2000-05-30 00:00:00,CONGELATORE NR . 49385 : CARCINOMA SPINOCELLUL...,34,9,349
...,...,...,...,...,...
13736,2013-10-24 10:35:14,NEOPLASIA LOBULARE INTRAEPITELIALE A GRADO DI ...,50,4,504
13817,2013-10-25 09:50:00,MAMMELLA DX : CARCINOMA LOBULARE INFILTRANTE (...,50,4,504
9280,2014-01-24 14:11:22,DOPPIA NODULARITA ' LOBO SUPERIORE DX + MICRON...,34,9,349
13846,2014-02-13 08:51:32,CARCINOMA DUTTALE INFILTRANTE ( G2 ) DELLA MAM...,50,9,509


In [62]:
dfUnlabelled = dfUn[['text']]

In [63]:
dfUnlabelled

,text
1,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLI...
2,ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGM...
3,FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI A...
4,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...
5,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI E...
...,...
1592380,"RESIDUE , MINIME ESPRESSIONI DI MALIGNITA ' EP..."
1592381,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...
1592382,E . R . : TESSUTO PROSTATICO RESIDUO CON CARAT...
1592383,CARCINOMA A CELLULE SQUAMOSE ( G2 ) CON ESPRES...


In [64]:
c = collections.Counter(dfFinal.yf)
c.most_common()

[(509, 4962),
 (349, 3557),
 (504, 2303),
 (508, 646),
 (502, 557),
 (341, 527),
 (505, 420),
 (503, 311),
 (343, 293),
 (501, 240),
 (340, 77),
 (342, 32)]

In [65]:
counts = {k:0 for k in numbers}

In [66]:
texts = dfFinal.text.values

In [67]:
ys = dfFinal.yf.values 

In [68]:
corpus={'text':[], 'site':[]}

In [69]:
for i in reversed(range(len(ys))):
    if counts[ys[i]] < numbers[ys[i]]:
        corpus['text'].append(texts[i])
        corpus['site'].append(ys[i])
        counts[ys[i]] += 1

In [70]:
c = collections.Counter(corpus['site'])
c.most_common()

[(509, 292),
 (349, 191),
 (341, 139),
 (343, 78),
 (504, 63),
 (508, 62),
 (502, 36),
 (340, 26),
 (505, 21),
 (501, 13),
 (342, 11),
 (503, 10)]

In [71]:
count = collections.Counter(corpus['site'])

In [72]:
values = sorted(count.keys())

In [73]:
len(values)

12

In [74]:
wordVectorsText = dfUnlabelled.text.values

In [75]:
np.random.shuffle(wordVectorsText)

In [76]:
wordVectorsText[:10]

array(['A . POLIPI DEL COLON SIGMOIDEO - B . MICROPOLIPO DEL COLON SIGMOIDEO A : TRE POLIPI IPERPLASTICI , DUE ADENOMI TUBULARI CON DISPLASIA DI BASSO GRADO . - B : MUCOSA COLICA A NORMALE MORFOLOGIA . 1 . CINQUE FRAMMENTI POLIPOIDI COMPLESSIVAMENTE DI 1X1X0 . 5 CM - 2 . FRAMMENTO DI TESSUTO DI 0 . 2 CM DI DIAMETRO',
       'PIASTRINE 41 . 000 NELLA NORMA EMOGLOBINA E GLOBULI BIANCHI . IL MATERIALE È COSTITUITO DA OSSO COMPATTO CON IMPONENTE NEO - APPOSIZIONE DI OSTEOIDI ; GLI SPAZI MIDOLLARI APPIONO IN TRASFORMAZIONE FIBROSA CON PICCOLI FOCOLAI DI INFILTRAZIONE DI CARCINOMA CON PARZIALE DIFFERENZIAZIONE GHIANDOLARE . BIOPSIA OSTEOMIDOLLARE DI MM 18 .',
       'GOZZO PLURINODULARE : TIROIDECTOMIA TOTALE ADENOMA TRABECOLARE TIROIDEO A CELLULE DI HURTHLE ( LOBO DESTRO ) . STRUMA TIROIDEA COLLOIDO - CISTICA CON NOTE DI TIROIDITE CRONICA LINFOCITARIA ( LOBO SINISTRO ) . 1 - 3 E 6 - 9 ) LOBO DESTRO . 4 - 5 ) LOBO SINISTRO .',
       'A . FRUSTOLO DI CORNO LATERALE DESTRO B . FRUSTOLO DI BAS

# write

In [77]:
pickle.dump(corpus, open(fileSaveCorpus, 'wb'))
pickle.dump(values, open(fileSaveValues, 'wb'))
np.savetxt(fileSaveTrainWordVectors, wordVectorsText, delimiter=" ", fmt="%s")